In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
from datetime import datetime

In [4]:
DATASET_SIZE = 120
DATASET_IS_BALANCED = True


MAX_FEATURES = 20000        # max_features params for CountVectorizer

training_name = 'glove-cnn-{}_{}k_{}'.format(
    MAX_FEATURES,
    DATASET_SIZE,
    'bal' if DATASET_IS_BALANCED else 'imbal'
)

training_args_datetime = datetime(year=2023, month=12, day=22)
training_storing_folder = Path(training_name).resolve()

In [6]:
import pickle

import tensorflow as tf
import keras

# load the tf model
# either a end-to-end
# or build our own (by loading the vectorizer and the model)


text_vectorizer_path = Path.joinpath(training_storing_folder, "{}_{}_textvectorizer.pkl".format(
    training_name,
    training_args_datetime.strftime("%Y-%m-%d")
))
vectorizer_from_disk = pickle.load(open(text_vectorizer_path, 'rb'))
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=MAX_FEATURES,
    output_sequence_length=512)

vectorizer.set_weights(vectorizer_from_disk['weights'])

model_path = Path.joinpath(training_storing_folder, "{}_{}_model.keras".format(
    training_name,
    training_args_datetime.strftime("%Y-%m-%d")
))
model = keras.models.load_model(model_path)

end_to_end_model_path = Path.joinpath(training_storing_folder, "{}_{}_end2end.keras".format(
    training_name,
    training_args_datetime.strftime("%Y-%m-%d")
))

end_to_end_model = keras.models.load_model(end_to_end_model_path)

print('\n\n')
print('Loaded text vectorizer from {}'.format(text_vectorizer_path))
print('Loaded model from {}'.format(model_path))
print('Loaded end to end model from {}'.format(end_to_end_model_path))




Loaded text vectorizer from /root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_textvectorizer.pkl
Loaded model from /root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_model.keras
Loaded end to end model from /root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_end2end.keras


In [7]:
end_to_end_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 vectorizer (TextVectorizat  (None, 512)               0         
 ion)                                                            
                                                                 
 model (Functional)          (None, 2)                 6808154   
                                                                 
Total params: 6808154 (25.97 MB)
Trainable params: 6808154 (25.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
vectorizer.get_vocabulary()[:20]

['',
 '[UNK]',
 'the',
 'and',
 'to',
 'a',
 'game',
 'i',
 'it',
 'of',
 'you',
 'is',
 'this',
 'in',
 'that',
 'for',
 'but',
 's',
 'with',
 't']

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 512, 300)             6000600   ['input_1[0][0]']             
                                                                                                  
 conv1d (Conv1D)             (None, 506, 128)             268928    ['embedding[0][0]']           
                                                                                                  
 conv1d_1 (Conv1D)           (None, 506, 128)             268928    ['embedding[0][0]']           
                                                                                              

In [13]:
# save the .keras model to SavedModel format
# which include a .pb file

# https://www.tensorflow.org/tutorials/keras/save_and_load#savedmodel_format

model.save(Path.joinpath(training_storing_folder, "{}_{}_model_savedmodel".format(
    training_name,
    training_args_datetime.strftime("%Y-%m-%d")
)))

INFO:tensorflow:Assets written to: /root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_model_savedmodel/assets


INFO:tensorflow:Assets written to: /root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_model_savedmodel/assets


In [22]:
# save the pretrained model to onnx

import tf2onnx
import onnx

onnx_model_path = Path.joinpath(training_storing_folder, "{}_{}_modelonly.onnx".format(
    training_name,
    training_args_datetime.strftime("%Y-%m-%d")
))

# onnx_model, _ = tf2onnx.convert.from_keras(
#     end_to_end_model,
#     input_signature=[tf.TensorSpec([None,1], dtype=tf.string, name='input_3')],
#     extra_opset='ai.onnx.contrib:1',
#     opset=13        # support onnxruntime >= 1.13.0
# )

# onnx.save(onnx_model, onnx_model_path)

In [23]:
print(training_storing_folder)
print('\n\n')
print(onnx_model_path)

/root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_120k_bal



/root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_modelonly.onnx


In [17]:
# https://github.com/onnx/tensorflow-onnx/issues/1867
# "probably there's no shared-name keyword for hash tables in TextVectorization layer."
# Therefore we only convert the model without the vectorizer (resulting in like huggingface, that the tokenizer is not included in onnx)

!python -m tf2onnx.convert --saved-model "glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_model_savedmodel" --output "glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_modelonly.onnx" --extra_opset ai.onnx.contrib:1 --opset 15

/root/miniforge3/envs/fyp-test2-wsl/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2023-12-22 16:39:46,832 - WARNING - tensorflow_text not installed. Model will fail to load if tensorflow_text ops are used.
2023-12-22 16:39:46,977 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2023-12-22 16:39:47,138 - INFO - Signatures found in model: [serving_default].
2023-12-22 16:39:47,138 - WARNING - '--signature_def' not specified, using first signature: serving_default
2023-12-22 16:39:47,138 - INFO - Output names: ['dense']
2023-12-22 16:39:47,138 - WARNING - Could not search for non-variable resources. Concrete function internal representation may have changed.
2023-12-22 16:39:47,550 - INFO - Using tensorflow=2.15.0, onnx=1.14.1, tf2onnx=1.15.1/37820d
2023-12-22 16:39:47,550 - INFO - Us

In [26]:
import onnxruntime as rt

sess = rt.InferenceSession(
    onnx_model_path,
    providers=['CPUExecutionProvider']
)

input_name = [inp.name for inp in sess.get_inputs()][0]     # only one input in this model
label_names = [label.name for label in sess.get_outputs()]  # it outputs the label and the probability

In [27]:
test_data = [['I like the game'], ["I do not like it."], ["It crashes when I just run on my pc."]]

In [43]:
# test inference

pred_keras = []
perd_onnx = []

for i in range(len(test_data)):
    # keras inference
    pred_keras.append(end_to_end_model.predict(test_data[i]))

    # onnx inference
    v_out = vectorizer(test_data[i])
    perd_onnx.append(sess.run(None, {"input_1": v_out.cpu().numpy().astype(np.int32)}))

print(pred_keras)
print(perd_onnx)

for i in range(len(test_data)):
    result_keras = pred_keras[i]
    result_onnx = perd_onnx[i][0]

    np.testing.assert_allclose(result_keras, result_onnx, rtol=1e-3, atol=1e-3)

1/1 [==============================] - 0s 19ms/step
[array([[0.34374118, 0.6562588 ]], dtype=float32), array([[0.8287184 , 0.17128158]], dtype=float32), array([[0.953253  , 0.04674701]], dtype=float32)]
[[array([[0.34371904, 0.65628093]], dtype=float32)], [array([[0.8287124 , 0.17128758]], dtype=float32)], [array([[0.9532483, 0.0467517]], dtype=float32)]]
